In [6]:
def test_train_val_split_80_10_10(img_root, ano_root):
    
    import shutil
    import os
    import random 

    # Set the percentages of images to use for testing and validation
    test_pct = 0.1
    val_pct = 0.1

    # Set the directories for the train, validation, and test data
    train_img = "app/train_img"
    train_ano = "app/train_ano"
    val_img = "app/val_img"
    val_ano = "app/val_ano"
    test_img = "app/test_img"
    test_ano = "app/test_ano"

    # Create the train, validation, and test directories
    for directory in [train_img, train_ano, val_img, val_ano, test_img, test_ano]:
        if not os.path.exists(directory):
            os.makedirs(directory)

    # Get the list of image filenames
    image_filenames = [filename for filename in os.listdir(img_root) if filename.endswith(".jpg")]

    # Calculate the number of images for testing and validation based on the percentages
    num_images = len(image_filenames)
    num_test_images = int(num_images * test_pct)
    num_val_images = int(num_images * val_pct)

    # Randomly select images for testing and validation
    test_image_filenames = random.sample(image_filenames, num_test_images)
    remaining_image_filenames = list(set(image_filenames) - set(test_image_filenames))
    val_image_filenames = random.sample(remaining_image_filenames, num_val_images)

    # Iterate through the image data root directory
    for filename in os.listdir(img_root):
        if filename.endswith(".jpg"):
            filename=os.path.basename(filename)
            src_path = os.path.join(img_root, filename)
            annotation_filename = filename.replace(".jpg", ".png")
            src_ano_path = os.path.join(ano_root, annotation_filename)
        
            if filename in test_image_filenames:
                # Move the image to the test directory
                dst_path = os.path.join(test_img, filename)
                dst_ano_path = os.path.join(test_ano, annotation_filename)
            elif filename in val_image_filenames:
                # Move the image to the validation directory
                dst_path = os.path.join(val_img, filename)
                dst_ano_path = os.path.join(val_ano, annotation_filename)
            else:
                # Move the image to the train directory
                dst_path = os.path.join(train_img, filename)
                dst_ano_path = os.path.join(train_ano, annotation_filename)
        
            # Move the image and the corresponding annotation file
            shutil.copy(src_path, dst_path)
            if os.path.exists(src_ano_path):
                shutil.copy(src_ano_path, dst_ano_path)

    print("Splitting successful!")

In [7]:
test_train_val_split_80_10_10("app/images","app/converted_masks")

Splitting successful!


In [8]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=4096)]
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus),"Logical_GPUs")

1 Physical GPU, 1 Logical_GPUs


In [9]:
from keras_segmentation.models.segnet import mobilenet_segnet
model = mobilenet_segnet(n_classes=2 ,  input_height=448, input_width=576)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 448, 576, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 450, 578, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 224, 288, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 224, 288, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 224, 288, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 226, 290, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 224, 288, 32)      288 

In [10]:
model.train(
    train_images =  "app/train_img",
    train_annotations = "app/train_ano",
    checkpoints_path = "app/checkpoints" , epochs= 2, 
    batch_size=2,
    steps_per_epoch=4006,
    validate=True,
    val_images="app/val_img",
    val_annotations="app/val_ano", 
    val_batch_size=2,
    val_steps_per_epoch=500,
)

Verifying training dataset
Verifying validation dataset
Epoch 1/2
4006/4006 [==============================] - 696s 173ms/step - loss: 0.1941 - accuracy: 0.9324 - val_loss: 0.1320 - val_accuracy: 0.9506

Epoch 00001: saving model to app/checkpoints.00001
Epoch 2/2
4006/4006 [==============================] - 694s 173ms/step - loss: 0.1337 - accuracy: 0.9515 - val_loss: 0.1176 - val_accuracy: 0.9537

Epoch 00002: saving model to app/checkpoints.00002


In [13]:
def mask_conversion(inp_dict,mask):
    keys_list = list(inp_dict.keys())
    for i in keys_list:
        mask[mask == i] = inp_dict[i]
    return mask

2024-04-08 02:53:08.033746: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-04-08 02:53:08.073327: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-04-08 02:53:08.119852: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-04-08 02:53:08.164438: W tensorflow/stream_executor/gpu/

In [14]:
import os 
for i , j in enumerate(os.listdir("app/test_img")):
    z=cv2.imread(os.path.join("app/test_img",j))
    v=cv2.imread(os.path.join("app/test_ano",j.replace(".jpg",".png")))
    dic={1:255}
    predicted_img = model.predict_segmentation(z)
    decoded_predicted_img=mask_conversion(dic,predicted_img)
    decoded_mask=mask_conversion(dic,v)
    # Resize the intersection mask using cv2
    resized_decoded_predicted_img = cv2.resize(decoded_predicted_img, (600, 450), interpolation=cv2.INTER_NEAREST)
    cv2.imencode('.png', resized_decoded_predicted_img)[1].tofile("app/testout/"+j.replace(".jpg","")+".png")
    cv2.imencode('.png', decoded_mask)[1].tofile("app/testpre/"+j.replace(".jpg","")+".png")

In [15]:
import numpy as np
def calculate_iou(mask1, mask2):
    """
    Calculate Intersection over Union (IoU) between two segmentation masks.

    Parameters:
        mask1 (numpy.ndarray): First segmentation mask.
        mask2 (numpy.ndarray): Second segmentation mask.
        
    Returns:
        float: IoU value.
    """
    
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

In [16]:
lst=[]
for i, x in enumerate(os.listdir("app/test_ano")):
    y=os.path.join("app/testout",x)
    x=os.path.join("app/testpre",x)
    x=cv2.imread(x)
    y=cv2.imread(y)
    x=np.array(x,dtype=np.uint8)
    y=np.array(y,dtype=np.uint8)
    lst.append(calculate_iou(x,y))
print(sum(lst)/i)

0.831213589136257
